In [1]:
import numpy as np

In [2]:
# 数据加载
import pandas as pd
data = pd.read_csv('family_data.csv', index_col='family_id')
data.head()

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4


In [3]:
# data里面的是距离圣诞节前几天？ 
# 最小值是1，因此，在后续处理时，可以-1，来索引
data.min()

choice_0    1
choice_1    1
choice_2    1
choice_3    1
choice_4    1
choice_5    1
choice_6    1
choice_7    1
choice_8    1
choice_9    1
n_people    2
dtype: int64

### 数据预处理
* 每个家庭的人数 Family_Size
* 每个家庭的选择倾向 DESIRED
* 计算Performance Cost矩阵 pcost_mat
* 计算Accounting Cost矩阵 acost_mat

In [4]:
# 参数
N_DAYS = 100 # 安排天数
N_FAMILY = 5000 # 家庭ID个数
MIN_OCCUPANCY = 125 # 最小参观人数
MAX_OCCUPANCY = 300 #最大参观人数

In [5]:
# 每个家庭的人数
FAMILY_SIZE = data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4], dtype=int64)

In [6]:
# 每个家庭的安排意愿
DESIRE = data.iloc[:, data.columns!='n_people'].values-1
DESIRE

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]], dtype=int64)

#### Performance Cost

In [7]:
# Performance Cost
def pcost(choice_n, family_number):
    if choice_n == 0:
        return 0
    elif choice_n == 1:
        return 50
    elif choice_n == 2:
        return 50 + (9)*family_number
    elif choice_n == 3:
        return 100 + (9)*family_number
    elif choice_n == 4:
        return 200 + (9)*family_number
    elif choice_n == 5:
        return 200 + (18)*family_number
    elif choice_n == 6:
        return 300 + (18)*family_number
    elif choice_n == 7:
        return 300 + (36)*family_number
    elif choice_n == 8:
        return 400 + (36)*family_number
    elif choice_n == 9:
        return 500 + (36+199)*family_number
    else:
        return 500 + (36+398)*family_number

In [8]:
# choice 和 family_num对cost的影响
# for choice in range(10):
#     for family_num in range(2,9):
#         print('Choice:{}, Family_number:{}, Pcost:{}'.format(choice, family_num, pcost(choice,family_num)))

In [9]:
# 计算pcost_mat 每个家庭被满足各个choice分别是什么情况
# 5000 * 100
# 先初始化一个矩阵
pcost_mat = np.full(shape=(N_FAMILY, 100), fill_value=99999)
for i in range(N_FAMILY):
    # 计算每个家庭各个choice的情况
    f_num = data.loc[i, 'n_people']      # 家庭成员数
    # 以最坏的情况初始化pcost_mat
    pcost_mat[i, :] = pcost(10, f_num)    # 以else的情况初始化矩阵，接着再对choice0-9进行修正
    for choice in range(10):
        day = data.iloc[i,choice]       # 第几个choice 具体是第几天
        pcost_mat[i, day-1] = pcost(choice, f_num)

#### accounting penalty

In [10]:
# 计算acounting penalty， 与前一天和当天参观人数相关
acost_mat = np.zeros(shape=(MAX_OCCUPANCY+1, MAX_OCCUPANCY+1), dtype=np.float64)
for i in range(acost_mat.shape[0]):     # 当天人数
    for j in range(acost_mat.shape[1]):  # 前一天人数
        diff = abs(i-j)
        acost_mat[i,j] = max(0, (i-125)/400*i ** (0.5+diff/50.0))            

In [11]:
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

### 使用LP+MIP分阶段求解
* 先利用LP对绝大部分家庭进行规划
* 再利用MIP对LP没规划好的家庭进一步求解
* 汇总两个阶段的求解结果，得到最终的规划方案
* 微调结果

In [12]:
from ortools.linear_solver import pywraplp

In [13]:
def solveLP():
    # 线性规划优化器
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    # 决策变量，{family_id，第几个choice}
    x ={}
    
    # 每天有哪些家庭想去参观 (100*?)
    candidates = [[] for x in range(N_DAYS)]
    
    for family_id in range(N_FAMILY):
        for desired_day in DESIRE[family_id,:]:
            x[family_id, desired_day] = solver.BoolVar('x[%i,%i]' %(family_id, desired_day))  
            candidates[desired_day].append(family_id)
    
    
    # 每天参观人数 
    # 每天想去参观的家庭candidates(100*...) 乘 该天的决策变量x[family_id,day] 0/1
    daily_occupancy = [solver.Sum([x[family_id,day] * FAMILY_SIZE[family_id] \
                                   for family_id in candidates[day]]) \
                                   for day in range(N_DAYS)]
    
    # 每个家庭，在10个choice中出现的总数
    # 这个总数是需要做约束的，每个家庭只需要满足一天就可以
    family_presence = [solver.Sum(x[family_id, day] \
                      for day in DESIRE[family_id,:]) \
                      for family_id in range(N_FAMILY)]
    
    # 定义目标函数 preference cost部分
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] \
                                  for i in range(N_FAMILY) \
                                  for j in DESIRE[i, :]]  
                                  )
    
    # 目标：最小化preference_cost
    solver.Minimize(preference_cost)
    
    # 由于 accounting cost需要较小的前后人数差，
    # 因此可以人为添加约束
    for j in range(N_DAYS-1):
        # j:当天，j+1：前一天    0 1,  98 99
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1] <= 25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j] <= 25)
    
    # 每个家庭只能出现一次
    for i in range(N_FAMILY):
        solver.Add(family_presence[i] == 1)
        
    # 每天访问人数 125 - 300
    for j in range(N_DAYS):      # 0 99
        solver.Add(daily_occupancy[j] >= MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j] <= MAX_OCCUPANCY)
    
    result = solver.Solve()
    
    # result
    temp = [(i,j,x[i,j].solution_value()) for i in range(N_FAMILY)\
            for j in DESIRE[i,:] if x[i,j].solution_value() > 0]
    print(solver.Objective().Value())
    # 得到参观日期的安排
    df = pd.DataFrame(temp, columns=['family_id', 'day', 'result'])
    
    return df

In [14]:
%%time
result = solveLP()

73702.31696428571
Wall time: 12.5 s


In [15]:
# 可以发现共有5000个家庭，但却有5073个安排结果
result

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5068,4995,15,1.0
5069,4996,87,1.0
5070,4997,31,1.0
5071,4998,91,1.0


In [16]:
# 安排结果不是0/1
result.result.value_counts()

1.000000    4301
1.000000     627
0.750000       2
0.666667       2
0.666667       2
            ... 
0.750000       1
0.400000       1
0.400000       1
0.400000       1
0.947917       1
Name: result, Length: 144, dtype: int64

In [17]:
# 设置阈值
Theshold = 0.999
# 
assigned_df = result[result.result > Theshold].reset_index(drop=True)
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
4926,4995,15,1.0
4927,4996,87,1.0
4928,4997,31,1.0
4929,4998,91,1.0


In [18]:
# 确认这些家庭被安排
len(assigned_df.family_id.unique())

4931

In [19]:
assigned_df.family_id.values

array([   0,    1,    2, ..., 4997, 4998, 4999], dtype=int64)

In [20]:
# 安排地不是那么好的家庭
unassigned_df = result[(result.result<Theshold) & (result.result > 1-Theshold)]
unassigned_df

,family_id,day,result
59,59,38,0.25
60,59,14,0.75
241,240,32,0.75
242,240,56,0.25
264,262,31,0.50
...,...,...,...
4983,4912,8,0.40
4985,4914,38,0.60
4986,4914,43,0.40
5033,4961,53,0.75


In [21]:
# 以家庭聚合，计算每个家庭的分配结果，其实加起来是为1
unassigned_df.groupby('family_id')['result'].sum()

family_id
59      1.0
240     1.0
262     1.0
357     1.0
488     1.0
       ... 
4869    1.0
4886    1.0
4912    1.0
4914    1.0
4961    1.0
Name: result, Length: 69, dtype: float64

In [22]:
# 那都是哪些家庭呢？
unassigned = unassigned_df.family_id.unique()
unassigned

array([  59,  240,  262,  357,  488,  610,  630,  724,  926,  941,  961,
       1012, 1106, 1167, 1176, 1218, 1235, 1271, 1287, 1387, 1444, 1484,
       1536, 1610, 1611, 1653, 1720, 1791, 1977, 2004, 2101, 2135, 2229,
       2318, 2344, 2395, 2641, 2778, 2862, 2874, 3033, 3054, 3104, 3118,
       3228, 3278, 3280, 3348, 3470, 3504, 3642, 3649, 3786, 4045, 4077,
       4084, 4404, 4421, 4431, 4471, 4518, 4661, 4820, 4850, 4869, 4886,
       4912, 4914, 4961], dtype=int64)

In [23]:
# 因为已经有家庭安排好了，因此每天的约束，最大最小会有变化

In [24]:
data.n_people

family_id
0       4
1       4
2       3
3       2
4       4
       ..
4995    4
4996    2
4997    6
4998    5
4999    4
Name: n_people, Length: 5000, dtype: int64

In [25]:
print(FAMILY_SIZE)

[4 4 3 ... 6 5 4]


In [26]:
# 家庭人数
assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]

In [27]:
assigned_df

,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
4926,4995,15,1.0,4
4927,4996,87,1.0,2
4928,4997,31,1.0,6
4929,4998,91,1.0,5


In [28]:
data.iloc[4679,-1]

5

In [29]:
assigned_df[assigned_df.day == 0]

,family_id,day,result,family_size
65,66,0,1.0,8
204,205,0,1.0,2
214,215,0,1.0,2
219,220,0,1.0,2
322,325,0,1.0,2
...,...,...,...,...
4617,4679,0,1.0,5
4654,4716,0,1.0,3
4710,4772,0,1.0,3
4737,4799,0,1.0,2


In [30]:
# 为什么第一天和老师写的不一样？？？？

occupancy = assigned_df.groupby('day')['family_size'].sum().values
occupancy

array([290, 271, 294, 293, 263, 242, 223, 247, 273, 297, 288, 292, 275,
       250, 238, 272, 292, 292, 271, 248, 223, 244, 264, 291, 292, 296,
       273, 249, 234, 251, 278, 283, 252, 235, 205, 184, 202, 233, 253,
       231, 210, 183, 204, 229, 247, 281, 256, 223, 204, 198, 222, 248,
       255, 223, 208, 185, 173, 196, 219, 198, 174, 141, 124, 121, 149,
       170, 178, 158, 136, 123, 125, 135, 158, 185, 158, 135, 121, 125,
       142, 167, 186, 167, 138, 120, 128, 155, 174, 203, 177, 152, 130,
       122, 130, 158, 178, 158, 128, 125, 122, 124], dtype=int64)

### Stage Two

In [31]:
diff = MIN_OCCUPANCY - occupancy
min_occupancy = np.where(diff>0, diff, 0 )

In [32]:
min_occupancy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 1], dtype=int64)

In [33]:
max_occupancy = MAX_OCCUPANCY - occupancy
max_occupancy

array([ 10,  29,   6,   7,  37,  58,  77,  53,  27,   3,  12,   8,  25,
        50,  62,  28,   8,   8,  29,  52,  77,  56,  36,   9,   8,   4,
        27,  51,  66,  49,  22,  17,  48,  65,  95, 116,  98,  67,  47,
        69,  90, 117,  96,  71,  53,  19,  44,  77,  96, 102,  78,  52,
        45,  77,  92, 115, 127, 104,  81, 102, 126, 159, 176, 179, 151,
       130, 122, 142, 164, 177, 175, 165, 142, 115, 142, 165, 179, 175,
       158, 133, 114, 133, 162, 180, 172, 145, 126,  97, 123, 148, 170,
       178, 170, 142, 122, 142, 172, 175, 178, 176], dtype=int64)

In [34]:
def solveIP(unassigned, min_occupancy, max_occupancy):
    # 整数规划
    #solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # 决策变量，{family_id，第几个choice}
    x = {}
    
    # 每天有哪些家庭想去参观 (100*?)
    candidates = [[] for x in range(N_DAYS)]
    
    # 决策变量只有unassigned的家庭
    for family_id in unassigned:
        for desired_day in DESIRE[family_id,:]:
            x[family_id, desired_day] = solver.BoolVar('x[%i,%i]' %(family_id, desired_day))  
            candidates[desired_day].append(family_id)
    
    # 每天参观人数 
    # 每天想去参观的家庭candidates(100*...) 乘 该天的决策变量x[family_id,day] 0/1
    daily_occupancy = [solver.Sum([x[family_id,day] * FAMILY_SIZE[family_id] \
                                   for family_id in candidates[day]]) \
                                   for day in range(N_DAYS)]
    
    # 每个家庭，在10个choice中出现的总数
    # 这个总数是需要做约束的，每个家庭只需要满足一天就可以
    family_presence = [solver.Sum(x[family_id, day] \
                      for day in DESIRE[family_id,:]) \
                      for family_id in unassigned]

    
    # 定义目标函数 preference cost部分
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] \
                                  for i in unassigned \
                                  for j in DESIRE[i, :]])
    
    # 目标：最小化preference_cost
    solver.Minimize(preference_cost)
    
    # 由于 accounting cost需要较小的前后人数差，
    # 因此可以人为添加约束
    for j in range(N_DAYS-1):
        # j:当天，j+1：前一天    0 1,  98 99
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1] <= 25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j] <= 25)
    
    # 每个家庭只能出现一次
    for i in range(len(unassigned)):
        solver.Add(family_presence[i] == 1)
        
    # 每天访问人数 125 - 300
    for j in range(N_DAYS):      # 0 99
        solver.Add(daily_occupancy[j] >= min_occupancy[j])           # 约束变了
        solver.Add(daily_occupancy[j] <= max_occupancy[j])
    
    result = solver.Solve()
    
    # result
    temp = [(i,j,x[i,j].solution_value()) for i in unassigned\
            for j in DESIRE[i,:] if x[i,j].solution_value() > 0]
    
    print(solver.Objective().Value())
    # 得到参观日期的安排
    df = pd.DataFrame(temp, columns=['family_id', 'day', 'result'])
    
    return df

In [35]:
%%time
result2 = solveIP(unassigned, min_occupancy, max_occupancy=max_occupancy)
result2

1401.0
Wall time: 145 ms


,family_id,day,result
0,59,38,1.0
1,240,32,1.0
2,262,31,1.0
3,357,24,1.0
4,488,39,1.0
...,...,...,...
64,4869,59,1.0
65,4886,98,1.0
66,4912,17,1.0
67,4914,38,1.0


In [36]:
# 拼接两个阶段的结果
result = pd.concat([assigned_df.loc[:, assigned_df.columns!='family_size'], result2]).sort_values(by='family_id').reset_index(drop=True)
result = result[['family_id', 'day']]
result

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
4995,4995,15
4996,4996,87
4997,4997,31
4998,4998,91


### 计算cost

In [37]:
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1, dtype=np.int64)
    penalty = 0
    for (i, p) in enumerate(prediction):
        # 计算家庭人数
        n = FAMILY_SIZE[i]
        # 第i个家庭，p天访问时的cost
        #print(i,p)
        penalty += pcost_mat[i, p]
        # 当天人数
        daily_occupancy[p] += n
    return penalty, daily_occupancy

def acost(daily_occupancy):
    accounting_cost = 0
    num_out_of_range = 0  
    daily_occupancy[-1] = daily_occupancy[-2]        # 注意 
    for day in range(N_DAYS):
        n_pl = daily_occupancy[day+1] # 前一天的人数
        n = daily_occupancy[day] # 当天的人数
        # 判断是否超过承载范围
        num_out_of_range += (n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        # 计算accounting cost
        accounting_cost += acost_mat[n, n_pl]
    return accounting_cost, num_out_of_range

In [38]:
def cost_function(prediction):
    # 计算两个cost
    penalty, daily_occupancy = pcost(prediction)
    accounting_cost, num_out_of_range = acost(daily_occupancy)
    final_score = penalty + accounting_cost + num_out_of_range*999999999  # 如果超过了承载数量，则会有很大的惩罚
    return final_score

In [39]:
prediction = result.day.values
cost_function(prediction)

77606.49757303706

#### 上面寻找最优解的时候只用了pcost这个目标，而acost并没加进去，因此，接下来还可以做一些微调

In [40]:
def find_better(pred):
    fobs = np.argsort(FAMILY_SIZE)
    score = cost_function(pred)
    original_score = np.inf
    
    # 
    while score<original_score:
        original_score = score
        i=0
        for family_id in fobs:
            for pick in range(10):
                # 得到family_id在choice pick 的日期 day
                day = DESIRE[family_id, pick]
                # 该family的原有日期
                oldvalue = pred[family_id]
                # 将原有的oldvalue 替换现在的choice pick
                pred[family_id] = day                
                # 重新计算分数
                new_score = cost_function(pred)

                # 比较
                if new_score < score:
                    score = new_score
                else:
                    pred[family_id] = oldvalue
                print(score, end='\r')
            print(score, end='\r')
    print(score)

### 这种不加约束的穷举会导致daily_occupancy存在大于300的数，后续应该还是通过规划求解来优化accounting cost目标

In [41]:
%%time
# .copy防止改变原来的值
new = prediction.copy()
find_better(new)

IndexError: index 302 is out of bounds for axis 0 with size 301